# 🐺 TEST 1: Monday AI Edge - Is It Real?
## Claim: CIFR +4.39% avg on Mondays, IREN +4.14%, RCAT +3.02%

**Goal:** Prove or disprove with ACTUAL DATA

**Decision Rule:**
- If win rate > 60% AND avg return > 2% → BUILD IT
- If not → DELETE THIS NOTEBOOK

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

print("✅ Ready to test")

---
## Test CIFR Monday Performance

In [ ]:
def test_monday_edge(ticker, months=6):
    """Test if a ticker really moves up on Mondays"""
    
    stock = yf.Ticker(ticker)
    hist = stock.history(period=f'{months}mo')
    
    if len(hist) == 0:
        return None
    
    # Get day of week (0 = Monday)
    hist['DayOfWeek'] = hist.index.dayofweek
    
    # Monday returns (open to close)
    mondays = hist[hist['DayOfWeek'] == 0].copy()
    mondays['Return'] = ((mondays['Close'] - mondays['Open']) / mondays['Open']) * 100
    
    # All other days for comparison
    other_days = hist[hist['DayOfWeek'] != 0].copy()
    other_days['Return'] = ((other_days['Close'] - other_days['Open']) / other_days['Open']) * 100
    
    # Calculate stats
    monday_avg = mondays['Return'].mean()
    monday_wins = (mondays['Return'] > 0).sum()
    monday_total = len(mondays)
    monday_win_rate = (monday_wins / monday_total * 100) if monday_total > 0 else 0
    
    other_avg = other_days['Return'].mean()
    other_wins = (other_days['Return'] > 0).sum()
    other_total = len(other_days)
    other_win_rate = (other_wins / other_total * 100) if other_total > 0 else 0
    
    # Results
    print(f"\n{'='*60}")
    print(f"📊 {ticker} - MONDAY EDGE TEST")
    print(f"{'='*60}")
    
    print(f"\n🔵 MONDAYS (n={monday_total}):")
    print(f"   Average Return: {monday_avg:+.2f}%")
    print(f"   Win Rate: {monday_win_rate:.1f}% ({monday_wins}/{monday_total})")
    print(f"   Best: {mondays['Return'].max():+.2f}%")
    print(f"   Worst: {mondays['Return'].min():+.2f}%")
    
    print(f"\n⚪ OTHER DAYS (n={other_total}):")
    print(f"   Average Return: {other_avg:+.2f}%")
    print(f"   Win Rate: {other_win_rate:.1f}% ({other_wins}/{other_total})")
    
    # The verdict
    edge_exists = monday_avg > other_avg and monday_win_rate > 55
    
    print(f"\n{'='*60}")
    if edge_exists:
        print(f"✅ EDGE CONFIRMED: Mondays outperform by {monday_avg - other_avg:+.2f}%")
    else:
        print(f"❌ NO EDGE: Mondays are NOT better ({monday_avg:+.2f}% vs {other_avg:+.2f}%)")
    print(f"{'='*60}")
    
    # Show last 10 Mondays
    print(f"\n📅 LAST 10 MONDAYS:")
    for date, row in mondays.tail(10).iterrows():
        emoji = "✅" if row['Return'] > 0 else "❌"
        print(f"   {emoji} {date.strftime('%Y-%m-%d')}: {row['Return']:+.2f}%")
    
    return {
        'ticker': ticker,
        'monday_avg': monday_avg,
        'monday_win_rate': monday_win_rate,
        'other_avg': other_avg,
        'edge_exists': edge_exists,
        'sample_size': monday_total
    }

# TEST CIFR
cifr_result = test_monday_edge('CIFR', months=6)

---
## Test All Monday AI Tickers

In [ ]:
# Test the full list
monday_tickers = ['CIFR', 'IREN', 'RCAT', 'HIVE', 'RIOT', 'APLD', 'UUUU']

results = []

for ticker in monday_tickers:
    result = test_monday_edge(ticker, months=6)
    if result:
        results.append(result)

# Summary table
print("\n" + "="*80)
print("📊 MONDAY EDGE SUMMARY")
print("="*80)

df = pd.DataFrame(results)
df = df.sort_values('monday_avg', ascending=False)

print(f"\n{'Ticker':<8} {'Mon Avg':<10} {'Win Rate':<12} {'Other Avg':<12} {'Edge?':<8} {'N':<5}")
print("-" * 80)

for _, row in df.iterrows():
    edge_symbol = "✅" if row['edge_exists'] else "❌"
    print(f"{row['ticker']:<8} {row['monday_avg']:+6.2f}%    {row['monday_win_rate']:5.1f}%      {row['other_avg']:+6.2f}%      {edge_symbol:<8} {row['sample_size']:<5.0f}")

# Count how many have edge
edge_count = sum(df['edge_exists'])
total_count = len(df)

print("\n" + "="*80)
if edge_count >= 4:
    print(f"✅ MONDAY EDGE IS REAL: {edge_count}/{total_count} tickers show edge")
    print("\n💰 ACTION: Build monday_ai_scanner.py")
    print("   - Run every Friday 3 PM")
    print("   - Buy positions at 3:59 PM close")
    print("   - Sell Monday 10 AM if up 2%+")
else:
    print(f"❌ MONDAY EDGE IS QUESTIONABLE: Only {edge_count}/{total_count} tickers show edge")
    print("\n🗑️ ACTION: DELETE this notebook. Don't build the tool.")
print("="*80)

---
## 💰 IF EDGE EXISTS: Calculate Expected Returns

In [ ]:
# If edge exists, calculate what $10K would make

if len(results) > 0 and any(r['edge_exists'] for r in results):
    
    # Filter to only edge tickers
    edge_tickers = [r for r in results if r['edge_exists']]
    
    # Calculate expected return
    avg_monday_return = np.mean([r['monday_avg'] for r in edge_tickers])
    
    # With $10K capital, 4-6 positions
    position_size = 10000 / 4  # $2,500 per position
    num_positions = 4
    
    expected_gain_per_position = position_size * (avg_monday_return / 100)
    expected_total_gain = expected_gain_per_position * num_positions
    
    # Per month (4 Mondays)
    monthly_expected = expected_total_gain * 4
    
    print("\n" + "="*60)
    print("💰 EXPECTED RETURNS WITH $10K CAPITAL")
    print("="*60)
    
    print(f"\nAverage Monday Return: {avg_monday_return:+.2f}%")
    print(f"Position Size: ${position_size:,.0f} x {num_positions} positions")
    print(f"\nExpected per Monday: ${expected_total_gain:+,.2f}")
    print(f"Expected per Month (4 Mondays): ${monthly_expected:+,.2f}")
    print(f"Expected Annual: ${monthly_expected * 12:+,.2f}")
    
    print(f"\n✅ If this works, $10K → ${10000 + (monthly_expected * 12):,.0f} in 1 year")
    
else:
    print("\n❌ No edge found. Don't trade this strategy.")

---
## 🎯 DECISION POINT

**Look at the results above:**

- ✅ **4+ tickers show edge** → Build monday_ai_scanner.py, use it Friday
- ❌ **< 4 tickers show edge** → DELETE this notebook, find better strategy

**Don't build what doesn't work.**